In [3]:
import tensorflow as tf

import numpy as np
import os
import time

2022-11-23 15:47:12.335787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
path_to_file = './data/shakespeare.txt'

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 5458199 characters


In [6]:
# Take a look at the first 250 characters in text
# print(text[:18000])
text = text[18000:]
# print(text[-1000:])
text = text[:-1000]
# print(text[-100:])

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

84 unique characters


In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-11-23 15:47:18.223650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[57, 58, 59, 60, 61, 62, 63], [80, 81, 82]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(5439199,), dtype=int64, numpy=array([68, 68, 75, ..., 60,  2, 70])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

l
l
s
 
t
h
e
 
t
i


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'l' b'l' b's' b' ' b't' b'h' b'e' b' ' b't' b'i' b'm' b'e' b',' b'\n'
 b' ' b' ' b'A' b'n' b'd' b' ' b's' b'e' b'e' b' ' b't' b'h' b'e' b' '
 b'b' b'r' b'a' b'v' b'e' b' ' b'd' b'a' b'y' b' ' b's' b'u' b'n' b'k'
 b' ' b'i' b'n' b' ' b'h' b'i' b'd' b'e' b'o' b'u' b's' b' ' b'n' b'i'
 b'g' b'h' b't' b',' b'\n' b' ' b' ' b'W' b'h' b'e' b'n' b' ' b'I' b' '
 b'b' b'e' b'h' b'o' b'l' b'd' b' ' b't' b'h' b'e' b' ' b'v' b'i' b'o'
 b'l' b'e' b't' b' ' b'p' b'a' b's' b't' b' ' b'p' b'r' b'i' b'm' b'e'
 b',' b'\n' b' '], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'lls the time,\n  And see the brave day sunk in hideous night,\n  When I behold the violet past prime,\n '
b" And sable curls all silvered o'er with white:  \n  When lofty trees I see barren of leaves,\n  Which e"
b"rst from heat did canopy the herd\n  And summer's green all girded up in sheaves\n  Borne on the bier w"
b'ith white and bristly beard:\n  Then of thy beauty do I question make\n  That thou among the wastes of '
b'time must go,\n  Since sweets and beauties do themselves forsake,\n  And die as fast as they see others'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'lls the time,\n  And see the brave day sunk in hideous night,\n  When I behold the violet past prime,\n'
Target: b'ls the time,\n  And see the brave day sunk in hideous night,\n  When I behold the violet past prime,\n '


In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  21760     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  87125     
                                                                 
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [44]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b". Why, I say nothing.\n  CLOWN. Marry, you are the wiser man; for many a man's tongue shakes\n    out "

Next Char Predictions:
 b'Nly[UNK]V!I5mSf3PY)\'n\n\'eV`8_>NcKYog\'!AeDF y1p-;QVz<9f `-WgX\'RV|E">IzQZ4s1RW cU[UNK];ChlZpyJyF}j!p]X6UZqTwKX2'


In [45]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [46]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.443584, shape=(), dtype=float32)


In [47]:
tf.exp(example_batch_mean_loss).numpy()

85.079315

In [48]:
model.compile(optimizer='adam', loss=loss)

In [49]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

b'Nly[UNK]V!I5mSf3PY)\'n\n\'eV`8_>NcKYog\'!AeDF y1p-;QVz<9f `-WgX\'RV|E">IzQZ4s1RW cU[UNK];ChlZpyJyF}j!p]X6UZqTwKX2'

In [50]:
history = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
841/841 [==============================] - 1320s 2s/step - loss: 1.8996
Epoch 2/20
 11/841 [..............................] - ETA: 21:47 - loss: 1.4735

KeyboardInterrupt: 

In [51]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [52]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [53]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO: O, he dream like devend'd.

            Enter A RICHALD] LUDIUS and LARDERSS, SCEPETHUS and this we driak, so our head of his weep
    Most before will take the wade of herefora, stoal days,
    Then you have aur less thee. Women of feast, and garlenest,
  And he beast be beauty, then fath bade own
was hast.
  ECHIS. Why lotery. I! Mastress, a leftive her.
  RECERDION. O, untill untext your queen, and a struetht is ever. Fellome, and his beabes,
    Where showest have puct from the restions strok.
  THIRDISE. Chates! Hold your body small hevers hand to him! Did our intle.
    Gobt, bring himmer? Choole her son, so would are not at
    shad when you may give me th' cately hid here. Thence, RIS RORIZAR Swall.

  BOLANTIM SENANY. This is it
  Least good bread prizents; for his outhorns?
Ander FORDALLUS, with Make hot shall be grief,
  I so queen of no but, so Ander sweet presene.
  FOLD. WILL, DIAMARD SIR TROBUS
  MACBEY. We hav' dead. Go doing in more Liaca.
    Terct these hath n

In [54]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO: CIPIUS and PARIA, about flore. Vere hath days and on too\n    Recessain humbly be drunk to convey thee wear him to deng her?\n    It is as willing noth. Comf roye to Pallotite\n\n  FIALCLUS. Whre had us, maight and in he ser their strange,\n  POHILLE, the infuccessius convition the prine eye\n    But there must patch thy jealon. I dight firmina,\n    Will poiet it thou admignt depold. Nor nature,\n    And man chose make stay me, If when is a jesticle.\n  Whils he't you thy reply hims? Sir, at hand'd wit, lose\n                        [The intess, and to help\n\nROUSEBLENA. When it would mark be dy\n    Unselordap out of our vileasus and rinculious, ship. Where might\n    What should so not, arail again eyes?\n\n                                                                                      Exit\n  ANT My oft, what it we shall did behone ot,\n    Which the worse faster with at my crown, A-bidling, coils for a liptle be foir will serve with they?\n  Friman CIRI

In [55]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [56]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:  What is stirents.
    Befold my youths, and gentle undoning the beath ot of my
    Th' regain. Har
